# Classification pipeline using GEE classifiers

Full glacier-snow-cover-mapping classification pipeline for Sentinel-2 TOA, Sentinel-2 SR, and Landsat 8/9 images. 

__Requirements:__
1. Google Earth Engine (GEE) account: used to query imagery and the DEM (if no DEM is provided). Sign up for a free account [here](https://earthengine.google.com/new_signup/). 

2. Google Drive folder, where output snow cover statistics will be saved.

## Define image search settings and paths

In [ ]:
import os
import ee
import geemap
import sys

# -----Define paths
# path to glacier-snow-cover-mapping
code_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/glacier-snow-cover-mapping/'
# Google Drive folder for outputs
# Note: Make sure this folder already exists and is the only folder in "My Drive" with that name. 
# Otherwise, multiple folders with the same name will be created!
out_folder = 'snow_cover_exports'

# -----Import pipeline utilities
sys.path.append(os.path.join(code_path, 'functions'))
import pipeline_utils_GEE as utils

# -----Define image search settings
# Date and month ranges (inclusive)
date_start = '2013-05-01'
date_end = '2025-05-07'
month_start = 6
month_end = 11
# Minimum fill portion of the AOI (0–100), used to remove images after mosaicking by day. 
min_aoi_coverage = 70
# Whether to mask clouds using the respective cloud mask via the geedim package
mask_clouds = True

## Authenticate and/or Initialize Google Earth Engine (GEE)

Replace the project ID with your GEE project. Default = `ee-{GEE-username}`

In [ ]:
project_id = "ee-raineyaberle"

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()
    ee.Initialize(project=project_id)

## Select the Area of Interest (AOI) from the GLIMS dataset

This cell will plot the GLIMS dataset on a map. To find a glacier, click on the wrench in the upper right toolbox of the map, and use the "Inspector" to click on a polygon and view the its properties. Right click on the "glac_id" property to highlight and then copy. Replace the `glac_id` variable below with your selected site. 

In [ ]:
# Create a Map
Map = geemap.Map()

# Add the GLIMS dataset to the map
glims = ee.FeatureCollection('GLIMS/20230607')
Map.addLayer(glims, {'color': 'blue', 'opacity':  0.5}, 'GLIMS/20230607')

# Select your study site
glac_id = 'G211100E60420N'
aoi = glims.filter(ee.Filter.eq('glac_id', glac_id))

# Merge all geometries to use as the AOI 
aoi = aoi.union().geometry()

# Add AOI to the map
Map.addLayer(aoi, {'color': 'orange', 'opacity': 0.8}, 'AOI')
Map.centerObject(aoi)

# Display the map
Map

## Load the Digital Elevation Model (DEM)

Default: use the ArcticDEM Mosaic where there is > 90 % coverage. Otherwise, use the NASADEM. For sites that use the ArcticDEM Mosaic, elevations are reprojected to the EGM96 geoid to match the vertical datum of NASADEM. 

In [ ]:
# Query GEE for DEM
dem = utils.query_gee_for_dem(aoi)

# Add DEM to map
minMax = dem.reduceRegion(reducer=ee.Reducer.minMax(),
                          geometry=aoi, 
                          scale=30,
                          maxPixels=1e9,
                          bestEffort=True)
elev_min = minMax.get('elevation_min')
elev_max = minMax.get('elevation_max')
print(f'Elevation range = {int(elev_min.getInfo())} - {int(elev_max.getInfo())} m')
# based on the "terrain" palette from matplotlib
palette = ['#333399', '#0d7fe5', '#00be90','#55dd77','#c6f48e','#e3db8a','#aa926b','#8e6e67','#c6b6b3','#ffffff']
Map.addLayer(dem, {'palette': palette, 'min' : elev_min, 'max': elev_max}, 'DEM')


## Split the date range into separate years

If the GEE computations take too long (> ~12 h), the final export times out. 
Splitting the date range and running each separately helps to mitigate time-out. 

In [ ]:
date_ranges = utils.split_date_range_by_year(date_start, date_end, month_start, month_end)
for date_range in date_ranges:
    print(date_range)

## Run the classification pipeline for each image dataset

### Sentinel-2 Top of Atmosphere (TOA)

In [ ]:
# Define dataset-specific params
dataset = "Sentinel-2_TOA"
resolution = 10

# Run the workflow for each year in the date range separately 
for date_range in date_ranges:
    print('\n', date_range)

    # Query GEE for imagery
    image_collection = utils.query_gee_for_imagery(dataset, aoi, date_range[0], date_range[1], month_start, month_end, 
                                                   min_aoi_coverage, mask_clouds)

    # Classify image collection
    classified_collection = utils.classify_image_collection(image_collection, dataset)

    # Calculate snow cover statistics, export to Google Drive
    stats = utils.calculate_snow_cover_statistics(classified_collection, dem, aoi, scale=resolution, out_folder=out_folder,
                                                  file_name_prefix=f"{glac_id}_{dataset}_snow_cover_stats_{date_range[0]}_{date_range[1]}")


### Sentinel-2 Surface Reflectance

In [ ]:
# Define dataset-specific params
dataset = "Sentinel-2_SR"
resolution = 10

# Run the workflow for each year in the date range separately 
for date_range in date_ranges:
    print('\n', date_range)

    # Query GEE for imagery
    image_collection = utils.query_gee_for_imagery(dataset, aoi, date_range[0], date_range[1], month_start, month_end, 
                                                   min_aoi_coverage, mask_clouds)

    # Classify image collection
    classified_collection = utils.classify_image_collection(image_collection, dataset)

    # Calculate snow cover statistics, export to Google Drive
    stats = utils.calculate_snow_cover_statistics(classified_collection, dem, aoi, scale=resolution, out_folder=out_folder,
                                                  file_name_prefix=f"{glac_id}_{dataset}_snow_cover_stats_{date_range[0]}_{date_range[1]}")

### Landsat 8/9

In [ ]:
# Define dataset-specific params
dataset = "Landsat"
resolution = 30

# Run the workflow for each year in the date range separately 
for date_range in date_ranges:
    print('\n', date_range)

    # Query GEE for imagery
    image_collection = utils.query_gee_for_imagery(dataset, aoi, date_range[0], date_range[1], month_start, month_end, 
                                                   min_aoi_coverage, mask_clouds)

    # Classify image collection
    classified_collection = utils.classify_image_collection(image_collection, dataset)

    # Calculate snow cover statistics, export to Google Drive
    stats = utils.calculate_snow_cover_statistics(classified_collection, dem, aoi, scale=resolution, 
                                                  file_name_prefix=os.path.join(f"{glac_id}_{dataset}_snow_cover_stats_{date_range[0]}_{date_range[1]}"))